Credits to Renu Khandelwal for some of the codes
https://medium.com/datadriveninvestor/generative-adversarial-network-gan-using-keras-ce1c05cfdfd3

Hello Eveyone~! This is your friendly neighborhood programmer, Seraph, about to challenge a type of Neural Network called a Generative Adversarial Network or GAN for short.

In this Notebook, we will be doing a test to see if a GAN network will be able to recreate, or probably, overcome, renowned artists' skilled paintings and artworks.

Then, without further ado, let's begin!

<h1> Read and Load Data </h1>
First, we need to read the datasets and prepare them for feeding into the NN. We would only need the images as we will be developing a GAN.

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import sys
from tqdm import tqdm

Here, since the Kaggle notebook has a limited RAM of about 16GB, we will be resizing our images to 256x256 to minimize the RAM usage, then, we'll turn the RGB values into the range of (-128, 128) by subtracting 128 from all values.

Furthermore, to save on RAM, we will turn the input getter into a GENERATOR that yields values only when needed.

In [ ]:
from skimage.transform import resize
#print(os.listdir("../input/best-artworks-of-all-time"))
w, h = 300, 300
print("Reading all CSV files...")
root_dir = "../input/best-artworks-of-all-time/"
images_dir = root_dir + 'images/images/'

#Get all authors
df = pd.read_csv(root_dir + "artists.csv")
df.replace(' ', '_', regex=True, inplace=True)
all_authors = list(df.name.values)
all_paintings = list(df.paintings)

from random import choice as rc
from random import randint as ri

def read_input(n):
    for _ in range(n):
        path = 'nonexistent'
        while not os.path.exists(path):
            auth = rc(all_authors)
            n = ri(1, all_paintings[all_authors.index(auth)])
            path = images_dir + auth + '/' + auth+'_'+str(n)+'.jpg'
        #print('Current Author: %s\tNumber: %i' %(auth, n), file=sys.stderr) #For debugging
        image = plt.imread(path)
        new_image = resize(image, (w, h), anti_aliasing=True)
        new_image = new_image/256
        yield new_image.flatten()

<h1>Network Creation</h1>
Now, we have read a part of the whole data. We will be needing the other artworks for further training, but I will not cover it in this Notebook.

Next, we would be declaring our Generator and Discriminator Networks. Since our input image are all resized to 512x512x3 (that is width x height x RGB) pixels, the output of the generator should also be 512x512x3.

In [ ]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Input, LeakyReLU
from tensorflow.keras.optimizers import Adam

def adam_optimizer():
    return Adam(lr=0.00005, beta_1=0.5)

def create_generator():
    generator = Sequential()
    generator.add(Dense(units=512, input_dim=100))
    generator.add(LeakyReLU(0.2))
    generator.add(Dense(units=1024))
    generator.add(LeakyReLU(0.2))
    generator.add(Dense(units=w*h*3, activation='sigmoid'))
    generator.compile(loss='binary_crossentropy', optimizer=adam_optimizer())
    return generator

g = create_generator()
g.summary()

def create_discriminator():
    discriminator=Sequential()
    discriminator.add(Dense(units=1024,input_dim=w*h*3))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
    discriminator.add(Dense(units=512))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
    discriminator.add(Dense(units=256))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dense(units=1, activation='sigmoid'))
    
    discriminator.compile(loss='mse', optimizer=adam_optimizer())
    return discriminator

d = create_discriminator()
d.summary()

Next, we will be declaring our GAN function

In [ ]:
def create_gan(discriminator, generator):
    discriminator.trainable=False
    gan_input = Input(shape=(100,))
    x = generator(gan_input)
    gan_output= discriminator(x)
    gan= Model(inputs=gan_input, outputs=gan_output)
    gan.compile(loss='binary_crossentropy', optimizer='adam')
    return gan
test = create_gan(d,g)
test.summary()
del test, d, g

<h1>Verification</h1>
To further see the results of the training, we will need to view a randomized image by the generator. Thus, we need this function to do this for us.

In [ ]:
def plot_generated_images(epoch, generator, examples=20, dim=(5,4), figsize=(5,4), to_file=False):
    noise= np.random.normal(loc=0, scale=1, size=[examples, 100])
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(examples,w,h,3)
    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i], interpolation='nearest')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('gan_generated_image %d.png' %epoch)
    if to_file:
        for i in range(examples):
            fname = 'image_%i.jpg' %i
            plt.imsave(fname, generated_images[i])


Finally, we'll be doing the Training! But first, we need to create a function that loads our data into the Network.

In [ ]:
def load_data(n=8):
    x_train = np.array([x for x in read_input(n)])
    y_train = np.array([1 for x in range(n)])
    return x_train, y_train

<h1>Training</h1>
And then, Define and call the function to train our Neural Networks!

In [ ]:
def training(epochs=1, batch_size=16):
    # Creating GAN
    generator= create_generator()
    discriminator= create_discriminator()
    gan = create_gan(discriminator, generator)
    previous_batch, _ = load_data(batch_size)
    for e in range(1,epochs+1 ):
        print("Epoch %d" %e)
    
        #Loading the data
        #To increase the number of samples, we load new data for every epoch
        X_train, y_train = load_data(batch_size)
        batch_count = X_train.shape[0] / batch_size
    
        for _ in tqdm(range(batch_size)):
        #generate  random noise as an input  to  initialize the  generator
            noise= np.random.normal(0,1, [batch_size, 100])
            
            # Generate fake MNIST images from noised input
            generated_images = generator.predict(noise)
            
            # Get a random set of  real images
            image_batch = X_train
            
            #Construct different batches of  real and fake data 
            #print(image_batch, file=sys.stderr)
            #print(image_batch.shape, generated_images.shape, file=sys.stderr)
            if len(image_batch.shape) != len(generated_images.shape):
                print('Error in shape!')
                image_batch = previous_batch
            else:
                previous_batch = image_batch
            X= np.concatenate([image_batch, generated_images])
            
            # Labels for generated and real data
            y_dis=np.zeros(2*batch_size)
            y_dis[:batch_size]=1.0
            
            #Pre train discriminator on  fake and real data  before starting the gan. 
            discriminator.trainable=True
            discriminator.train_on_batch(X, y_dis)
            
            #Tricking the noised input of the Generator as real data
            noise= np.random.normal(0,1, [batch_size, 100])
            y_gen = np.ones(batch_size)
            
            # During the training of gan, 
            # the weights of discriminator should be fixed. 
            #We can enforce that by setting the trainable flag
            discriminator.trainable=False
            
            #training  the GAN by alternating the training of the Discriminator 
            #and training the chained GAN model with Discriminator’s weights freezed.
            gan.train_on_batch(noise, y_gen)
            
        if e%25==0:# e == 1 or e % 5 == 0 or e:
            plot_generated_images(e, generator)
    #We will now save a full size image batch of the test
    plot_generated_images(0, generator, 20, (5,4), (5,4), True)
    #Now, we'll save our models to preserve them
    #generator.save('generator.h5')
    #discriminator.save('discriminator.h5')
    #gan.save('GAN.h5')

training(4000, 16)